# Курсовая работа. Тетрадь 2

## 2.1 Общие предустановки для работы с векторными моделями и загрузка 400 самых частотных названий профессий

In [23]:
# Кусочек кода, необходимый для решения проблем с сертификатом при загрузке данных
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

In [24]:
import gensim
import zipfile
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans, AgglomerativeClustering

Загружаю csv-файл, содержащий в себе информацию о 400 самых поппулярных профессиях базы. Добавляю новый столбец, в котором слова предобработаны для подачи их в модель (содержат частеречные теги)

In [25]:
top400 = pd.read_csv('/Users/annalevina/top400.csv', low_memory=False, sep=";") 

In [26]:
cat_named = {"0": "Армия", "1": "Промышленность", "2": "Сельское хозяйство", "3": "Лесное хозяйство", "4": "Строительство", "5": "Транспорт и связь", "6": "Торговля, заготовки и общественное питание", "7": "Жилищное и коммунальное хозяйство", "8": "Просвещение, наука, искусство, печать", "9": "Здравоохранение", "10": "Государственные учреждения, партийные и общественные организации", "11": "Нераспределенные по отраслям народного хозяйства и неточно указавшие отрасль", "12": "Служители культа"}

In [27]:
top400["cat_named"] = top400["cat"].apply(lambda x: cat_named[str(x)])

In [28]:
top400["word"] = top400['prof_after_women_postproc']+'_NOUN'

In [29]:
top400

,prof_after_women_postproc,cat,freq,cat_named,word
0,рабочий,1,32404,Промышленность,рабочий_NOUN
1,колхозник,2,20062,Сельское хозяйство,колхозник_NOUN
2,бухгалтер,10,18379,"Государственные учреждения, партийные и общест...",бухгалтер_NOUN
3,красноармеец,0,17690,Армия,красноармеец_NOUN
4,начальник,10,15213,"Государственные учреждения, партийные и общест...",начальник_NOUN
...,...,...,...,...,...
395,укладчик,1,110,Промышленность,укладчик_NOUN
396,каталь,1,110,Промышленность,каталь_NOUN
397,шахтер,1,107,Промышленность,шахтер_NOUN
398,рубщик,1,106,Промышленность,рубщик_NOUN


Для работы я выбрала 3 модели:
* НКРЯ 
* НКРЯ и Википедия за декабрь 2018
* Тайга

Работа с каждой моделью описана в отдельном блоке. Алгоритм работы для всех моделей аналогичен, поэтому я приведу более подробное описание для первой из них, а в остальных случаях ограничусь комментариями. 

## 2.2 Работа с моделью НКРЯ 

Скачиваю модель для русского языка, созданную на основе Национального корпуса русского языка (НКРЯ). Разархивирую в формате gensim-a

In [30]:
with zipfile.ZipFile('/Users/annalevina/Desktop/Универ/Магистратура/КУРСОВАЯ/НКРЯ/180.zip', 'r') as archive:
    stream = archive.open('180/model.bin')
    model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

Каждое слово с частеречным тегом подаю модели в цикле. Создаю словарь, в который отправляю слова, содержащиеся в модели, аттрибуты полученных векторов нумерую, присваиваю названия столбцам таблицы. Модель работает не со всеми названиями профессй, некоторые она не знает и из-за этого возникает  `KeyError`. Чтобы понимать, какие слова не могут быть предствалены как векторы (и пропущены моделью), при этой ошибке название профессии распечатывается.

In [31]:
words_vectors = pd.DataFrame([])
words_no_vectors = []
for word in top400["word"]:
    try:
        vector = model.word_vec(word)
        vector_dict = dict(zip(["attr" + str(num) for num in range(1, 301)], vector))
        vector_dict['word'] = word
        words_vectors = words_vectors.append(vector_dict, ignore_index=True)
    except KeyError:
        print(word)
        words_no_vectors.append(word)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  """


крестьянин-единоличник_NOUN
учительник_NOUN
техник-строитель_NOUN
коновозчик_NOUN
портн_NOUN
экономист-плановик_NOUN
ветсанитар_NOUN
домохозяй_NOUN
бракер_NOUN
крепильщик_NOUN
инженер-механик_NOUN
мобилизованный_NOUN
электрослесарь_NOUN
строгальщик_NOUN
инженер-конструктор_NOUN
инженер-строитель_NOUN
заготовщик_NOUN
учащаяий_NOUN
откатчик_NOUN
счетовод-кассир_NOUN
слесарь-механик_NOUN
секретарь-машинист_NOUN
прессовщик_NOUN
ездовой_NOUN
крестьян_NOUN
пимокат_NOUN
инженер-электрик_NOUN
дояр_NOUN
шлифовщик_NOUN
инженер-технолог_NOUN
слесарь-водопроводчик_NOUN
оперуполномоченный_NOUN
слесарь-инструментальщик_NOUN
плановик-экономист_NOUN
бухгалтер-ревизор_NOUN
прач_NOUN
пилоправ_NOUN
рамщик_NOUN
техник-нормировщик_NOUN
инженер-экономист_NOUN
монаш_NOUN
гуртоправ_NOUN
сверловщик_NOUN
арматурщик_NOUN
техник-конструктор_NOUN
инженер-химик_NOUN
десятник-строитель_NOUN
путеобходчик_NOUN
таксировщик_NOUN
вагоновожатый_NOUN
техник-механик_NOUN
штамповщик_NOUN
сырьевщик_NOUN
ломовой_NOUN
масленщик

In [32]:
words_vectors

,attr1,attr10,attr100,attr101,attr102,attr103,attr104,attr105,attr106,attr107,...,attr91,attr92,attr93,attr94,attr95,attr96,attr97,attr98,attr99,word
0,0.016639,1.913702,-0.934010,-1.707090,0.581804,-0.198912,-0.540220,3.614760,-0.373158,0.045577,...,0.976734,-0.173329,2.140803,-0.917508,-1.230901,-1.754601,-1.866635,-0.054754,0.796853,рабочий_NOUN
1,0.302769,1.041670,0.383878,-1.673300,-0.511423,-1.300838,0.193410,4.292620,1.781163,0.623018,...,1.109783,-0.106138,1.461375,0.418790,-0.824394,-1.815287,-2.176465,-0.118531,0.424016,колхозник_NOUN
2,-4.025567,-1.850026,0.199329,-0.149826,0.839867,-0.136252,0.352311,-0.142961,2.077302,-0.855545,...,-0.389467,-1.633253,-1.511830,1.275657,-1.107764,-1.613538,-0.817821,-0.971441,-0.841680,бухгалтер_NOUN
3,-0.234275,1.059857,0.969475,-1.615372,-0.378846,0.017759,-0.484774,1.022244,1.343390,2.561270,...,0.263591,-1.839009,0.584237,-1.639306,0.732968,-0.862381,-0.153403,-2.166710,0.922480,красноармеец_NOUN
4,-0.645020,-0.929079,-1.665457,-0.984864,1.887612,-0.361507,0.922778,1.515141,0.588563,0.372951,...,0.784352,-0.632420,-2.094697,1.885234,0.861793,0.260878,1.063307,-1.437121,-2.030204,начальник_NOUN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,-0.189876,-0.172660,-0.493085,0.217774,-0.055832,-0.094660,0.339598,0.167400,0.296277,-0.019902,...,0.672204,-0.075362,0.244109,0.077628,-0.465397,-0.332008,-0.019528,0.080993,-0.050238,укладчик_NOUN
340,-0.144947,0.126909,0.044198,-0.206276,0.250655,-0.168895,0.124247,0.118331,0.203592,0.073412,...,0.148383,-0.073292,0.244786,-0.226155,-0.041879,-0.338907,0.072896,-0.287456,0.188677,каталь_NOUN
341,2.824451,0.759943,-1.656304,0.871027,-1.390498,-2.104200,-0.844311,1.248761,3.296548,0.231710,...,-0.036406,-2.548056,0.971626,0.208793,0.344264,-2.908785,-2.294670,-1.116414,-0.142310,шахтер_NOUN
342,-0.102257,-0.339801,-0.521564,-0.053371,-0.089819,-0.081760,0.126543,0.046140,0.314877,0.104311,...,0.049900,-0.224168,0.010239,0.185751,0.097104,-0.040395,-0.166803,-0.128047,-0.314482,рубщик_NOUN


Присваиваю `words_vectors` как значение для переменной `top400_vectors`. Полученную таблицу преобразую в массив, поскольку именно такой тип данных необходим для кластерного анализа.

In [34]:
top400_vectors = words_vectors
top400_vectors_attr = np.array(top400_vectors[["attr" + str(num) for num in range(1, 301)]])

Рассматриваю алгоритм кластеризации `K-Means`. Количество кластеров = 13

In [35]:
clusters_kmeans = KMeans(n_clusters=13, random_state=0).fit_predict(top400_vectors_attr)
clusters_kmeans

array([ 5,  5, 11,  7,  0,  5, 11,  3,  1,  3, 11,  1,  5, 11,  5,  1, 11,
        5,  5,  1,  3,  1, 10, 11,  9, 10,  7,  1, 10,  1, 12,  7, 11,  3,
        5,  3, 11,  1,  1,  5, 11, 11,  9,  5,  1,  5,  3,  3,  3, 11,  0,
        3, 11,  7,  7,  6,  3,  3,  7,  6,  3,  7,  1,  5,  5,  3,  3,  6,
        3,  3,  3, 11,  1,  1,  3,  3,  1,  0, 10,  3,  3,  1,  3,  1,  3,
        5, 11,  3,  3,  3,  3,  6,  3,  2,  3,  5,  3,  3,  3,  3,  3,  7,
        3,  3,  3,  3,  3,  0,  1,  0,  3,  6,  2,  3,  0,  5,  3,  3,  0,
        1,  5,  3,  5,  3, 10,  3, 10,  6,  3, 12,  3,  3,  3,  3,  7, 11,
        4,  3,  9,  3,  3,  3,  3,  3,  3, 12,  3,  3,  2,  3, 11,  7,  3,
        7,  3,  3,  3, 12,  3,  7,  3,  0,  3, 11,  3,  3,  0,  0,  3,  5,
        6,  2,  4, 11,  3,  3,  3,  5,  3,  3,  4,  9,  0,  0,  0,  3,  3,
        8,  0,  3,  3,  3,  3,  3,  6,  7,  5,  3,  2,  3,  3,  3,  9, 10,
        3,  3,  3,  3,  1,  3,  3,  3,  7,  5,  3,  3,  3,  3,  3,  7,  7,
        3,  3,  3,  1,  8

Добавляю столбец в таблицу со словами, которые могут быть представлены в виде векторов. 

In [36]:
top400_vectors["cluster_kmeans_NKRL"] = clusters_kmeans

Рассматриваю алгоритм кастеризации `Agglomerative Clustering`. Количество кластеров = 13

In [37]:
clusters_agglomerative = AgglomerativeClustering(n_clusters=13, linkage='average', affinity='cosine').fit_predict(top400_vectors_attr)
clusters_agglomerative

array([12,  2,  5,  0,  3,  5,  3,  5,  5,  5,  3,  2,  5,  3, 12,  5,  3,
        5,  5,  5,  5,  5,  6,  3,  6,  6,  0,  5,  6,  5,  1,  0,  3,  5,
        5,  5,  0,  6,  5,  5,  3, 12,  6, 12,  2,  5,  5,  3, 12, 12,  0,
        0,  3,  0,  0,  9,  2,  5,  0,  2,  5,  5,  5,  5,  5,  5,  3,  9,
        5,  5,  3,  3,  3,  5,  5,  2,  5,  0,  6,  5,  5,  5,  5,  5,  3,
        5,  4,  5,  2, 12,  5,  9,  5, 12,  5,  5,  2,  3, 12,  5,  5,  0,
        5,  2,  5,  6,  5,  0,  5,  0,  5,  9,  2,  5,  0,  5,  2,  2,  5,
        5,  5,  3,  2,  0,  6,  5,  6,  6,  3,  1,  2,  5,  1,  5,  2,  3,
        6,  5,  6,  5,  2,  5, 11,  5,  5,  1,  5,  5,  5,  5, 12,  0,  4,
        0,  5,  3,  5,  1,  5,  5,  2,  5,  5,  4,  2,  2,  0,  0,  5, 12,
        6, 12,  4,  6,  5,  6,  6,  2,  5,  5,  4,  6,  0,  0,  0,  5,  2,
        3,  0,  5,  5,  5,  6,  2,  6,  0, 12,  6,  2,  2,  0,  5,  6,  6,
        5,  3,  5,  2,  5,  6,  0, 12,  5, 12,  4,  5,  5,  1,  5,  0,  0,
        5,  2,  6,  5,  3

Этот столбец также добавляю в таблицу `top400_vectors`.

In [38]:
top400_vectors["cluster_agglomerative_NKRL"] = clusters_agglomerative

Добавляю столбцы с результатами кластеризации в таблицу `top400`.

In [39]:
top400_out = pd.merge(top400, top400_vectors[['word', 'cluster_kmeans_NKRL', 'cluster_agglomerative_NKRL']],
                      how='left', on='word')

## 2.3 Работа с моделью Тайга

Скачиваю модель для русского языка, созданную на основе открытого и структурированного корпуса русского языка. Разархивирую в формате gensim-a

In [40]:
with zipfile.ZipFile('/Users/annalevina/Desktop/Универ/Магистратура/КУРСОВАЯ/Тайга/185.zip', 'r') as archive:
    stream = archive.open('model.bin')
    model_Taiga = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

Подаю названия профессий с частеречными тегами модели.

In [41]:
words_vectors_Taiga = pd.DataFrame([])
words_no_vectors_Taiga = []
for word in top400["word"]:
    try:
        vector = model_Taiga.word_vec(word)
        vector_dict = dict(zip(["attr" + str(num) for num in range(1, 301)], vector))
        vector_dict['word'] = word
        words_vectors_Taiga = words_vectors_Taiga.append(vector_dict, ignore_index=True)
    except KeyError:
        print(word)
        words_no_vectors_Taiga.append(word)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  """


крестьянин-единоличник_NOUN
учительник_NOUN
техник-строитель_NOUN
коновозчик_NOUN
технорук_NOUN
ветфельдшер_NOUN
портн_NOUN
табельщик_NOUN
осмотрщик_NOUN
сортировщик_NOUN
экономист-плановик_NOUN
ветсанитар_NOUN
домохозяй_NOUN
бракер_NOUN
крепильщик_NOUN
инженер-механик_NOUN
браковщик_NOUN
мобилизованный_NOUN
строгальщик_NOUN
инженер-конструктор_NOUN
инженер-строитель_NOUN
заготовщик_NOUN
модельщик_NOUN
учащаяий_NOUN
откатчик_NOUN
счетовод-кассир_NOUN
слесарь-механик_NOUN
секретарь-машинист_NOUN
ездовой_NOUN
крестьян_NOUN
пимокат_NOUN
инженер-электрик_NOUN
инженер-технолог_NOUN
слесарь-водопроводчик_NOUN
оперуполномоченный_NOUN
слесарь-инструментальщик_NOUN
плановик-экономист_NOUN
бухгалтер-ревизор_NOUN
сигналист_NOUN
лесообъездчик_NOUN
прач_NOUN
пилоправ_NOUN
рамщик_NOUN
воентехник_NOUN
техник-нормировщик_NOUN
инженер-экономист_NOUN
монаш_NOUN
гуртоправ_NOUN
сверловщик_NOUN
арматурщик_NOUN
техник-конструктор_NOUN
инженер-химик_NOUN
десятник-строитель_NOUN
вальцовщик_NOUN
путеобходчик_N

In [42]:
words_vectors_Taiga

,attr1,attr10,attr100,attr101,attr102,attr103,attr104,attr105,attr106,attr107,...,attr91,attr92,attr93,attr94,attr95,attr96,attr97,attr98,attr99,word
0,0.063455,0.097537,0.020513,0.022959,0.478973,0.078407,0.377432,-0.142065,0.172441,-0.162961,...,0.112455,-0.067703,-0.011255,-0.146223,0.056332,-0.029045,-0.244479,0.114332,0.109309,рабочий_NOUN
1,0.078339,0.035631,0.132465,0.219033,0.395405,0.025878,0.416815,-0.217168,0.016888,-0.275503,...,0.351719,-0.163174,0.306959,-0.209274,-0.098197,0.057289,-0.339743,-0.073980,-0.217823,колхозник_NOUN
2,-0.096454,0.107843,0.009668,0.023590,0.365888,-0.010180,0.150978,0.129313,-0.074199,-0.000743,...,0.110979,0.002955,0.116200,-0.011777,0.118546,-0.213144,-0.353287,-0.301079,-0.132214,бухгалтер_NOUN
3,-0.244058,-0.053432,0.294390,0.086562,0.408053,-0.067889,-0.231863,-0.215144,-0.104172,0.041296,...,0.140638,-0.023264,0.176952,-0.110277,-0.013999,-0.170336,-0.102357,-0.017091,-0.252012,красноармеец_NOUN
4,0.034154,0.513612,0.114845,0.148746,0.261864,-0.238959,0.142903,-0.204769,-0.321213,-0.024037,...,0.000575,0.030547,0.011645,-0.071013,-0.051164,-0.166272,-0.420361,-0.246961,0.115157,начальник_NOUN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,0.017001,0.172727,0.364308,0.575903,0.072305,-0.373137,-0.386927,-0.317535,-0.182228,-0.281522,...,0.115325,-0.129148,-0.171373,0.014580,-0.082958,-0.181391,-0.271014,-0.190362,-0.079261,корреспондент_NOUN
331,0.266385,0.020071,-0.039378,0.167142,0.007813,0.259462,0.438942,-0.309874,-0.379338,0.367948,...,0.063747,0.026205,-0.410513,-0.090860,0.463265,-0.144263,-0.663835,-0.055504,-0.037412,укладчик_NOUN
332,-0.242532,0.041868,0.049679,-0.169199,0.463980,-0.127410,0.345850,0.035181,0.062654,-0.266735,...,0.164772,-0.166844,-0.095994,0.037702,0.249249,0.239309,-0.107521,-0.273212,-0.097322,шахтер_NOUN
333,0.260270,0.120021,0.016249,-0.047280,0.876647,-0.135168,0.672673,0.074681,0.083130,0.173018,...,0.200792,0.173200,0.212788,0.332767,-0.038622,-0.056180,0.111206,-0.127164,-0.175646,рубщик_NOUN


In [43]:
top400_vectors_Taiga = words_vectors_Taiga
top400_vectors_attr_Taiga = np.array(top400_vectors_Taiga[["attr" + str(num) for num in range(1, 301)]])

Алгоритм кластеризации `K-Means`.

In [47]:
clusters_kmeans_Taiga = KMeans(n_clusters=13, random_state=0).fit_predict(top400_vectors_attr_Taiga)
clusters_kmeans_Taiga

array([ 0,  0,  7,  3,  9,  6,  9,  2,  7,  7,  9,  7,  6,  9,  7,  1,  9,
        6,  6,  1, 10,  7,  8,  9,  5,  8,  0,  2,  8,  7, 12,  3,  2, 10,
       10,  2,  3,  7,  1,  7,  2,  2,  5,  0,  7,  6,  7,  2,  2,  2,  3,
        0,  9,  3,  3,  5, 11,  7,  3,  5,  9,  3,  2,  7,  6,  2,  7,  2,
        1,  6,  9,  9,  2,  7,  2, 11,  2,  3,  8, 10,  7,  2,  7,  1,  9,
        6,  4,  2, 11, 11,  6,  2,  1, 11, 10,  6, 10,  9, 11,  6,  2,  3,
        6, 11,  7,  7,  7,  3,  7,  3,  1,  2, 11,  6,  0,  7,  0,  5,  3,
        7,  6,  3, 11,  0,  8,  6,  8,  5, 12,  2,  2,  3,  4,  4,  3,  5,
       11,  0, 10,  7, 10, 12,  7,  7,  3,  7,  0,  3,  4,  0, 12,  2,  1,
       12,  6,  2,  5,  3, 10,  8, 10, 11,  3,  3,  0,  5,  0,  4,  4,  7,
        4,  6,  0,  1,  7,  4,  5,  3,  3,  3,  2,  7,  4,  3,  2,  7,  6,
        4, 11,  5,  0,  0,  4, 11,  6,  3,  7,  5,  8,  2,  2,  7,  5,  2,
        7,  3,  0,  3,  0,  4,  2,  1, 12,  0,  3,  2,  7,  5,  3,  7,  9,
        7, 12, 11,  7, 11

In [48]:
top400_vectors_Taiga["cluster_kmeans_Taiga"] = clusters_kmeans_Taiga

Алгоритм кастеризации `Agglomerative Clustering`.

In [49]:
clusters_agglomerative_Taiga = AgglomerativeClustering(n_clusters=13, linkage='average', affinity='cosine').fit_predict(top400_vectors_attr_Taiga)
clusters_agglomerative_Taiga

array([ 1,  1,  1,  0,  5,  1,  5,  1,  1,  1,  6,  1,  1,  5,  1,  1,  5,
        1,  1,  1,  1,  1,  5,  5,  1,  5,  2,  1,  5,  1,  3,  0,  5,  1,
        0,  1,  1,  1,  1,  1, 10,  1,  1,  1,  1,  1,  1,  0,  0,  1,  0,
        2,  5,  0,  0,  5,  1,  1,  0,  1, 12,  0,  0,  1,  1,  1,  1,  5,
        1,  1,  5,  5,  0,  1,  1,  1,  0,  0,  5,  1,  1,  1,  1,  1,  1,
        1,  6, 12,  1, 12,  1,  5,  1, 12,  1,  1,  1,  5, 12,  1,  0,  0,
        1,  1,  1,  1,  1,  0,  1,  0,  1,  5,  1,  1,  0,  1,  1,  1,  0,
        1,  1,  7,  1,  2,  5,  1,  5,  5,  3,  0,  0,  0,  4,  4,  1,  1,
       12,  1,  1,  1,  0,  3,  1,  1,  1,  1,  2,  0,  4,  0,  3,  0,  1,
        3,  1,  0,  5,  0,  1,  6,  1,  1,  0,  5,  1,  5,  2,  4,  5,  1,
        4,  4,  1,  1,  1,  4,  1,  0,  0,  0,  0,  1,  5,  0,  0,  1,  1,
        4,  1,  5, 10,  1,  1,  1,  1,  0,  1,  1,  6,  0,  0,  1,  1,  1,
        1,  0,  1,  1,  2,  6,  0,  1,  3,  2,  0,  0,  1,  8,  1,  1,  5,
        1,  3,  1,  1,  1

In [50]:
top400_vectors_Taiga["cluster_agglomerative_Taiga"] = clusters_agglomerative_Taiga

Добавляю столбцы с результатами кластеризации в таблицу `top400`.

In [51]:
top400_out = pd.merge(top400_out, top400_vectors_Taiga[['word', 'cluster_kmeans_Taiga', 'cluster_agglomerative_Taiga']],
                      how='left', on='word')

## 2.4 Работа с моделью НКРЯ + Wikipedia

Скачиваю модель, в основе которой - дамп русской Википедии за декабрь 2018 года. Разархивирую в формате gensim-a

In [52]:
with zipfile.ZipFile('/Users/annalevina/Desktop/Универ/Магистратура/КУРСОВАЯ/Wiki/182.zip', 'r') as archive:
    stream = archive.open('182/model.bin')
    model_Wiki = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

Обработанные названия профессий(тегами частей речи) подаю модели.

In [53]:
words_vectors_Wiki = pd.DataFrame([])
words_no_vectors_Wiki = []
for word in top400["word"]:
    try:
        vector = model_Wiki.word_vec(word)
        vector_dict = dict(zip(["attr" + str(num) for num in range(1, 301)], vector))
        vector_dict['word'] = word
        words_vectors_Wiki = words_vectors_Wiki.append(vector_dict, ignore_index=True)
    except KeyError:
        print(word)
        words_no_vectors_Wiki.append(word)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  """


крестьянин-единоличник_NOUN
учительник_NOUN
техник-строитель_NOUN
коновозчик_NOUN
ветфельдшер_NOUN
портн_NOUN
табельщик_NOUN
осмотрщик_NOUN
экономист-плановик_NOUN
ветсанитар_NOUN
домохозяй_NOUN
бракер_NOUN
крепильщик_NOUN
браковщик_NOUN
мобилизованный_NOUN
строгальщик_NOUN
инженер-конструктор_NOUN
инженер-строитель_NOUN
заготовщик_NOUN
учащаяий_NOUN
откатчик_NOUN
счетовод-кассир_NOUN
слесарь-механик_NOUN
секретарь-машинист_NOUN
прессовщик_NOUN
ездовой_NOUN
крестьян_NOUN
пимокат_NOUN
инженер-электрик_NOUN
дояр_NOUN
шлифовщик_NOUN
инженер-технолог_NOUN
слесарь-водопроводчик_NOUN
оперуполномоченный_NOUN
слесарь-инструментальщик_NOUN
плановик-экономист_NOUN
такелажник_NOUN
бухгалтер-ревизор_NOUN
лесообъездчик_NOUN
прач_NOUN
пилоправ_NOUN
рамщик_NOUN
техник-нормировщик_NOUN
инженер-экономист_NOUN
монаш_NOUN
гуртоправ_NOUN
сверловщик_NOUN
арматурщик_NOUN
техник-конструктор_NOUN
инженер-химик_NOUN
автослесарь_NOUN
десятник-строитель_NOUN
путеобходчик_NOUN
таксировщик_NOUN
вагоновожатый_NOUN


In [54]:
words_vectors_Wiki

,attr1,attr10,attr100,attr101,attr102,attr103,attr104,attr105,attr106,attr107,...,attr91,attr92,attr93,attr94,attr95,attr96,attr97,attr98,attr99,word
0,-0.345069,-0.394684,0.069989,0.049652,-0.277877,0.013882,0.153408,0.077828,-0.113149,0.059131,...,-0.024828,-0.093746,-0.393818,0.148985,-0.022838,-0.291340,-0.101754,0.497088,0.074406,рабочий_NOUN
1,-0.494410,0.127278,-0.109530,0.236413,-0.653939,0.298139,0.382290,0.190506,-0.506847,0.232627,...,-0.195604,0.078454,-0.180486,0.425868,0.144145,-0.083897,-0.165513,0.609362,-0.293181,колхозник_NOUN
2,-0.003305,-0.438756,0.050146,-0.287629,-0.185487,-0.286608,-0.017928,0.184129,-0.400744,0.201877,...,0.201709,0.141929,-0.092114,-0.089716,0.055092,-0.372259,-0.223968,0.094823,-0.279633,бухгалтер_NOUN
3,-0.382692,0.058631,0.307757,0.152779,-0.670603,0.034919,-0.040635,-0.068242,-0.355073,-0.076552,...,-0.346296,-0.128670,-0.278174,-0.181278,0.031588,0.123025,-0.131326,0.061561,0.008603,красноармеец_NOUN
4,-0.270656,-0.183626,-0.499086,-0.040675,-0.145153,0.126696,0.032018,0.138784,-0.050604,0.120049,...,-0.089841,0.152843,-0.382965,-0.072365,-0.174975,-0.141930,-0.074462,-0.064587,0.274553,начальник_NOUN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332,-0.315857,0.078594,-0.312349,-0.505375,-0.512883,0.155633,-0.319513,0.198449,-0.121758,0.091643,...,-0.053291,0.341710,-0.645056,0.113787,0.309302,0.362419,-0.028643,0.069764,0.087434,комбриг_NOUN
333,0.104657,-0.466811,-0.051073,-0.052043,0.289599,0.056739,0.132407,0.181968,-0.310116,-0.026770,...,-0.128578,-0.218513,-0.178754,-0.414671,-0.473639,-0.119676,-0.337243,0.119371,-0.614522,корреспондент_NOUN
334,0.228611,0.505503,-0.233729,0.082990,-0.532796,0.008331,-0.116013,0.282876,0.155715,0.340101,...,0.102495,-0.066012,-0.578341,-0.167559,0.412643,-0.241327,0.432331,0.101246,-0.199895,укладчик_NOUN
335,-0.136664,-0.099164,0.136010,0.195646,-0.463443,0.079602,0.031201,0.267109,-0.154611,0.308782,...,-0.172004,0.052205,-0.063309,0.071679,-0.059498,-0.083172,-0.083474,0.296715,-0.021942,шахтер_NOUN


In [55]:
top400_vectors_Wiki = words_vectors_Wiki
top400_vectors_attr_Wiki = np.array(top400_vectors_Wiki[["attr" + str(num) for num in range(1, 301)]])

Первый алгоритм кластеризации - `K-Means`.

In [56]:
clusters_kmeans_Wiki = KMeans(n_clusters=13, random_state=0).fit_predict(top400_vectors_attr_Wiki)
clusters_kmeans_Wiki

array([ 7, 10,  3,  6,  0,  7,  0, 12,  4,  3,  9,  4,  7,  0,  7,  3,  0,
        7,  7,  2,  7,  3,  9,  0,  1,  9, 12,  2,  9,  1,  5,  6,  0,  7,
        7, 12,  0,  1,  2,  4, 12, 12, 11,  8, 10,  7,  4,  3,  3, 12,  6,
        9,  0, 12,  6, 11,  7,  3,  9, 10,  0, 12, 12,  7,  7,  7,  3, 12,
        2,  7,  0,  0,  3,  4, 12,  7,  2,  6,  9,  3,  3,  2,  4,  2,  0,
        7, 11,  3,  7,  3,  7, 12,  2,  7,  7,  7,  7,  0,  3,  7, 12, 12,
        7, 11,  4,  3,  3, 12,  2,  6, 12, 12,  7,  7, 12,  4,  8, 10,  6,
        1,  7, 12,  7,  8, 11,  7,  9,  1,  3,  5, 12, 12, 12,  0, 11,  2,
        1,  3,  8, 12,  4, 12,  5,  4,  1, 12,  3,  9, 12, 11, 12, 12,  0,
        7,  5,  7, 12, 10, 12,  7,  9,  7, 10,  6,  6,  3,  8, 11,  8, 11,
        0,  4, 11,  7,  8,  2,  4, 11,  1,  6,  6,  6, 12, 10, 11,  6, 12,
        7,  7, 11, 10,  0, 12,  8,  0,  7,  7,  6,  4, 11,  9, 12, 12, 12,
        1, 10,  2,  4, 12,  8,  2,  9, 12, 12,  2,  5, 12,  6, 12, 10, 11,
        2,  4,  0,  4,  5

In [57]:
top400_vectors_Wiki["cluster_kmeans_Wiki"] = clusters_kmeans_Wiki

Второй алгоритм - `Agglomerative Clustering`.

In [58]:
clusters_agglomerative_Wiki = AgglomerativeClustering(n_clusters=13, linkage='average', affinity='cosine').fit_predict(top400_vectors_attr_Wiki)
clusters_agglomerative_Wiki

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  3,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  2,  1,  1,  2,  1,  1,  2,  1,  8,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  2,  4,  1,  1,  1,  1,  4,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  2,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  1,  1, 10,  1,  1,  1,
        1,  3,  1,  1,  4,  1,  1,  1,  4,  1,  1,  1,  1,  4,  1,  1,  1,
        1,  2,  1,  2,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  4,  1,  2,  1,  2,  2,  1,  8,  1,  0,  1,  7,  2,  1,
        1,  4,  4,  1,  1,  1,  8,  1,  1,  1,  1,  5,  1,  2,  1,  1,  1,
        1,  8,  1,  1,  1,  1,  1,  3,  1,  1,  1,  1,  1,  4,  2,  4,  2,
        1,  1,  2,  1,  4,  1,  1,  2,  1,  1,  1,  1,  1,  1,  2,  1,  1,
        1,  1,  7,  1,  2,  1,  4,  1,  1,  1,  1,  1,  2,  2,  1,  1,  1,
        1,  1,  1,  1, 11,  4,  1,  5,  3,  1,  1,  8,  1,  1,  1,  1,  6,
        1,  1,  1,  1,  8

In [59]:
top400_vectors_Wiki["cluster_agglomerative_Wiki"] = clusters_agglomerative_Wiki

Добавляю столбцы с результатами кластеризации в таблицу `top400`. Формирую итоговую таблицу и выгружаю ее в формате `csv`.

In [65]:
top400_out = pd.merge(top400_out, top400_vectors_Wiki[['word', 'cluster_kmeans_Wiki', 'cluster_agglomerative_Wiki']],
                      how='left', on='word')
top400_out_full = top400_out[["prof_after_women_postproc", "cat", "cat_named", "freq",
                              "cluster_kmeans_NKRL", "cluster_agglomerative_NKRL",
                              "cluster_kmeans_Taiga", "cluster_agglomerative_Taiga",
                              "cluster_kmeans_Wiki", "cluster_agglomerative_Wiki"]]

In [67]:
top400_out_full.to_csv("Full_clusters.csv")

In [68]:
top_out_win = top400_out[["prof_after_women_postproc", "cluster_agglomerative_NKRL"]]

In [69]:
top_out_win.to_csv("Named_final_clusters.csv")

## 2.5 Сравнение результатов работы моделей. Выбор финальной модели.

Подгружаю из `sklearn.metrics` метод `adjusted_rand_score`, который оценит работу алгоритмов (их соответствие кластерам, размеченным мной вручную). 

In [298]:
from sklearn.metrics import adjusted_rand_score

В итоговой таблице `top400_out` встречаются отсутствующие значения, вызванные тем, что какая-то из моделей могла не узнать то или иное слово. Эти ячейки удаляю.

In [299]:
top400_out_cat = top400_out[~pd.isnull(top400_out["cat"]) & ~pd.isnull(top400_out["cluster_kmeans_NKRL"]) & ~pd.isnull(top400_out["cluster_agglomerative_NKRL"]) & ~pd.isnull(top400_out["cluster_kmeans_Taiga"]) & ~pd.isnull(top400_out["cluster_agglomerative_Taiga"]) & ~pd.isnull(top400_out["cluster_kmeans_Wiki"]) & ~pd.isnull(top400_out["cluster_agglomerative_Wiki"])]

Тестирую `K-Means` для НКРЯ

In [300]:
result_NKRL_kmeans = adjusted_rand_score(top400_out_cat['cat'], top400_out_cat['cluster_kmeans_NKRL'])
result_NKRL_kmeans

0.06201406843633836

`Agglomerative Clustering` для НКРЯ

In [301]:
result_NKRL_agglomerative = adjusted_rand_score(top400_out_cat['cat'], top400_out_cat['cluster_agglomerative_NKRL'])
result_NKRL_agglomerative

0.25604569375449066

`K-Means` для Тайги

In [302]:
result_Taiga_kmeans = adjusted_rand_score(top400_out_cat['cat'], top400_out_cat['cluster_kmeans_Taiga'])
result_Taiga_kmeans

0.2143495104416532

`Agglomerative Clustering` для Тайги

In [303]:
result_Taiga_agglomerative = adjusted_rand_score(top400_out_cat['cat'], top400_out_cat['cluster_agglomerative_Taiga'])
result_Taiga_agglomerative

0.20641358616357008

`K-Means` для корпуса НКРЯ и Википадии

In [304]:
result_Wiki_kmeans = adjusted_rand_score(top400_out_cat['cat'], top400_out_cat['cluster_kmeans_Wiki'])
result_Wiki_kmeans

0.16802147358657427

`Agglomerative Clustering` для корпуса НКРЯ и Википадии

In [305]:
result_Wiki_agglomerative = adjusted_rand_score(top400_out_cat['cat'], top400_out_cat['cluster_agglomerative_Wiki'])
result_Wiki_agglomerative

0.07245331579077169

In [ ]:
most.similar("проводник_NOUN")